# Make ner dataset CoNLL2003 format with ann file from yedda annotation tool

191018

BIOES format

Aotomate chunking with NLTK


Get CoNLL2003 format using yedda code(to make anns file)

However, sometimes there is critical errors because the difference between NLTK chunking tokenize and tagging token tokenize

We try 2 ways to get the CoNLL2003 format from ann file(yedda annotation tool result)



In [1]:
## put your input .ann folder 

folder_path = 'test'

In [2]:
import os, re, string
import nltk
from nltk import word_tokenize, pos_tag

from functions import *

In [3]:
# add space right after *] 
# add space right before [@
def _split(pattern):
    string = str(pattern.group())    
    if '[@' in string:
        return string.split('[')[0] + ' ' + '[@'
    elif '[$' in string:
        return string.split('[')[0] + ' ' + '[$'
    elif '*]' in string:
        return '*]' + ' ' + string.split(']')[1]


# add space if there is '.' in patter
def _split_comma(pattern):
    string = str(pattern.group())
    if '.' in string:
        return string.split('.')[0]

# Tagging!!!

In [4]:
file_list = os.listdir(folder_path)
print('doc %d ea' % (len(file_list)))
# ~_conll folder
conll_folder_path = folder_path + '_conll'
if not os.path.isdir(conll_folder_path): os.mkdir(conll_folder_path)

    
error_sentence_list = []  # save error sentence
for file_path in file_list:
#     filename = 'test.ann'
    
    ## 1. load
    sentences = []
    with open(os.path.join(folder_path, file_path), 'r', encoding='UTF-8') as f:
        for line in f.readlines():
            line = line.strip()
            sentences.append(line)
        
    conll_document = []
    ## 2. line running
    for sentence in sentences:
        conll_sentence_with_chunking = []
        sentence = sentence.replace(" '", " ' ")
        
        if sentence != '':
            ##############################################################################
            ## 1) data preprocessing
            ##############################################################################
            ## 1. '.' tokenize problem, split before&after '.'
            # 1.1. find patterns
            pattern_list = re.findall(r'\[[\@\$)].*?\#.*?\*\](?!\#)', sentence)
            for pattern in pattern_list:
                if '.' in pattern:
                    no_comma_pattern = pattern.replace('.', ' . ')
            # 1.2. if there is '.', add space before&after
                    sentence = sentence.replace(pattern, no_comma_pattern)
            # refind patterns
            pattern_list = re.findall(r'\[[\@\$)].*?\#.*?\*\](?!\#)', sentence)
            for pattern in pattern_list:
                if '[$' in pattern:
                    no_comma_pattern = pattern.replace('[$', '[@')
                    sentence = sentence.replace(pattern, no_comma_pattern)      
            ##############################################################################
            ## 2. tokenize problen solving
            ## 2.1. add space after tag('*]'') if no space agter
            taged_sentence_back     = re.sub(r'\*\][^\ ]', _split, sentence)
            ## 2.2. add space at before tag('[@' or '[$') if no space before tag
            taged_sentence_filtered = re.sub(r'[^\ ]\[[\@\$]', _split, taged_sentence_back)            
            ##############################################################################
            
            
            ##############################################################################
            ## 2) conll tagging
            ##############################################################################
            ## 1. make conll tagging format 
            conll_format = make_ann2conll(taged_sentence_filtered)
            ##############################################################################
            ## 2. make chunk format
            ## 2.1 restore sentence
            
            pure_line = re.sub(r'\[[\@\$)].*?\#.*?\*\](?!\#)', back2pure_line, taged_sentence_filtered)
            ## 2.2 make chunk
            chunking     = make_chunk(pure_line)
            ##############################################################################
            ## 3. check the the length between chunking and tagging is different or not
            ## 3.1. if the lengh is same, merge chunk and tagging
            if len(chunking) == len(conll_format):
                
                for i in range(len(chunking)):
                    conll_sentence_with_chunking.append(chunking[i] + [conll_format[i][-1]])
                # add black after sentence
                conll_sentence_with_chunking.append([''])
            
            ## 3.2. If the length between chunking and tagging is different
            else:
                try:
                    if len(chunking) != len(conll_format):
                        print('!!! - not possible with first way')

                        conll_format1 = make_ann2conll(sentence)
                        conll_format2 = get_tagging_token(sentence)
                        ##############################################################################
                        ## 2. make chunk format
                        ## 2.1 restore sentence
                        pure_line = make_pure_sentence_from_tagged(sentence)
                        ## 2.2 make chunk
                        chunking     = make_chunk(pure_line)
                        print('chunk : %d / conll1 : %d / conll1 : %d'%(len(chunking), len(conll_format1), len(conll_format2)))                    

                        if len(chunking) != len(conll_format2):
                            print('!!! - not possible with second way, Fix ann file with hand....')
                            error_sentence_list.append([file_path, sentence, pure_line, chunking, conll_format, len(chunking), len(conll_format)])
                        else:
                            print('!!!!! - second way, complete')
                            conll_format = conll_format2
                            # complete conversion, merge
                            for i in range(len(chunking)):
                                conll_sentence_with_chunking.append(chunking[i] + [conll_format[i][-1]])
                            # add black after sentence
                            conll_sentence_with_chunking.append([''])
                except:
                    print('Fix ann file with hand....')
                    error_sentence_list.append([file_path, sentence, pure_line, chunking, conll_format, len(chunking), len(conll_format)])

        # add doc_list after sentence end
        conll_document.append(conll_sentence_with_chunking)
    
    ####################################################################
    ## save
    ####################################################################
    ## save as document unit
    fila_name = file_path.split('.ann')[0]
    fw = open(os.path.join(conll_folder_path, fila_name  + '.conll'), 'w')
    # start document with file name
    fw.write('<<doc>> ' + fila_name +'\n\n')
    for conll_line in conll_document:
        for conll_token in conll_line:
            if len(conll_token) > 0:
                try:
                    fw.write('\t'.join(conll_token) + '\n')
                except:
                    pass
    fw.close()
    ####################################################################
    ####################################################################
print('Done, error : %d'%(len(error_sentence_list)))

doc 2 ea
Done, error : 0


In [5]:
print(len(error_sentence_list))
error_sentence_list

0


[]

In [6]:
idx = 0
get_error_information(error_sentence_list, idx)

No error!!
